<a href="https://colab.research.google.com/github/EjbejaranosAI/Deep-Learning/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install necesary packages

In [1]:
!pip install -q pandas
!pip install -q tensorflow
!pip install -q opencv-python
!pip install -q scikit-learn
!pip install -q wandb
!wandb login

     |████████████████████████████████| 1.9 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 56.1 MB/s 
     |████████████████████████████████| 168 kB 53.1 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 168 kB 61.6 MB/s 
     |████████████████████████████████| 166 kB 55.6 MB/s 
     |████████████████████████████████| 166 kB 50.4 MB/s 
     |████████████████████████████████| 162 kB 55.0 MB/s 
     |████████████████████████████████| 162 kB 50.0 MB/s 
     |████████████████████████████████| 158 kB 40.5 MB/s 
     |████████████████████████████████| 157 kB 5.3 MB/s 
     |████████████████████████████████| 157 kB 55.5 MB/s 
     |████████████████████████████████| 157 kB 48.5 MB/s 
     |████████████████████████████████| 157 kB 66.5 MB/s 
     |████████████████████████████████| 157 kB 57.7 MB/s 
     |████████████████████████████████| 157 kB 55.4 MB/s 
     |████████████████████████████████| 157 kB 55.4 MB/s 
     |████████████

# Download data

In [2]:
# Download dataset 256x256 
!curl -o /content/drive/MyDrive/Colab_Notebooks_upc/5_DL/Delivery_2/MAMe_data_256.zip https://storage.hpai.bsc.es/mame-dataset/MAMe_data_256.zip
# Download labels 
!curl -o /content/drive/MyDrive/Colab_Notebooks_upc/5_DL/Delivery_2/MAMe_metadata.zip https://storage.hpai.bsc.es/mame-dataset/MAMe_metadata.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  689M  100  689M    0     0  20.0M      0  0:00:34  0:00:34 --:--:-- 22.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  848k  100  848k    0     0   579k      0  0:00:01  0:00:01 --:--:--  579k


In [3]:
# Download dataset High-Resolution
#!curl -o /content/drive/MyDrive/Colab_Notebooks_upc/5_DL/Delivery_2/MAMe_data.zip https://storage.hpai.bsc.es/mame-dataset/MAMe_data.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  205G    0  869M    0     0  20.6M      0  2:49:13  0:00:42  2:48:31 23.7M^C


# Get GPU information

In [5]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# Import Libraries

```
# api_key = 190989b8377e79d615c691b45c63d5fcc4e5b29f 
```

In [17]:
import os
import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import random

import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib.rcsetup import validate_sketch

import tensorflow
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence

from tensorflow.keras.utils import to_categorical

In [15]:
# Get data
!cp /content/drive/MyDrive/Colab_Notebooks_upc/5_DL/Delivery_2/MAMe_data_256.zip .
!cp /content/drive/MyDrive/Colab_Notebooks_upc/5_DL/Delivery_2/MAMe_metadata.zip .

!unzip /content/MAMe_metadata.zip
!rm MAMe_metadata.zip

!unzip /content/MAMe_data_256.zip
!rm MAMe_data_256.zip

# Helper functions

In [20]:
# Dataloader
class Mame_dataloader(tf.keras.utils.Sequence):
    def __init__(self, df, x_col, y_col=None, batch_size=32, num_classes=None, shuffle=True):
        self.batch_size = batch_size
        self.df = df
        self.indices = self.df.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.x_col = x_col
        self.y_col = y_col
        self.on_epoch_end()

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        index = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(index)
        return X, y

    def on_epoch_end(self):
        if self.shuffle == True:
            np.random.shuffle(self.indices)

    def __get_data(self, index):
        
        X = []
        y = []
        
        img_batch = self.x_col[index].tolist()
        label_batch = self.y_col[index].tolist()
        
        for img_name, label in zip(img_batch, label_batch):
            img = cv2.imread(img_name) / 255.0
            #img = img.resize((224, 224))
            label = int(label)
            lb = np.zeros(self.num_classes)
            lb[label] = 1
    
            X.append(img)
            y.append(lb)
        X = np.array(X)
        
        return X, np.array(y)


def preprocessing_metadata(path_metadata, path_data, path_labels):
  print("")
  print("------------------Starting with pre-processing--------------------")
  print("")
        
  df = pd.read_csv(path_metadata)
  df['path'] = path_data
  df['path image'] = df['path'] + df['Image file']

  label_dataframe = pd.read_csv(path_labels)
  label_dataframe.head()

  header = ['target', 'label']
  label_dataframe.loc[-1] = label_dataframe.columns
  label_dataframe.columns = header
  #labels.sort_values('target',ascending=False)
  label_dataframe = label_dataframe.reset_index(drop=True)
  # Remove blank spaces
  label_dataframe['label'] = label_dataframe['label'].str.strip()
  label_dataframe.head()

  #Replace Medium names by labels 
  labels = label_dataframe['label'].values
  new_labels = label_dataframe['target'].values

  labels_dict = dict(zip(labels, new_labels))
  df['Medium'] = df['Medium'].str.strip()
  df['Medium'] = df.loc[:,'Medium'].replace(labels_dict) 
  print(df.head())
  print("")
  print("------------------Pre-processing finished--------------------")
  print("")    
  
  
  return df

def samplesByClass(dataframe,num_to_see:int):
  print("")
  print("-----------------Dataframe grouped by Medium class--------------------")
  print("")
  dataframe['count'] = 1
  num_by_class = dataframe.groupby('Medium').sum().reset_index()
  num = num_by_class.head(num_to_see)
  print(num.head())
  print("")
  return num


# Get the number of samples by subset
def subsetByClass(df):
  print("")
  print("-----------------Dataframe grouped by Subset class--------------------")
  print("")
  subset_by_class = df.groupby('Subset').sum().reset_index()
  print(subset_by_class.head())
  print("")
  

# Pre-processing data

In [22]:
path_data = '/content/data_256'
path_toy_data = '/content/MAMe_toy_dataset.csv'
path_labels = '/content/MAMe_labels.csv'
path_metadata = '/content/MAMe_dataset.csv'

# Generate dataframe with data ready to create subset list
df_metadata = preprocessing_metadata(path_metadata,path_data, path_labels)   
# Get the number of samples by medium class
samplesByClass(df_metadata, 29)

# Get the number of samples by subset class
subsetByClass(df_metadata)

# Generate dataframe for each subset
train, test, val = split_dataframe_subset(df_metadata)


------------------Starting with pre-processing--------------------

   Image file Medium                      Museum Museum-based instance ID  \
0  436018.jpg     17  Metropolitan Museum of Art                29.100.60   
1   11779.jpg     17  Metropolitan Museum of Art                 1982.373   
2   19022.jpg     17  Metropolitan Museum of Art                 2006.418   
3  435883.jpg     17  Metropolitan Museum of Art                 61.101.3   
4   10481.jpg     17  Metropolitan Museum of Art                    09.95   

  Subset   Width  Height  Product size  Aspect ratio               path  \
0  train  3144.0  3840.0    12072960.0        0.8187  /content/data_256   
1  train  1707.0  2136.0     3646152.0        0.7992  /content/data_256   
2  train  2845.0  3811.0    10842295.0        0.7465  /content/data_256   
3  train  3811.0  2916.0    11112876.0        1.3069  /content/data_256   
4  train  3811.0  2099.0     7999289.0        1.8156  /content/data_256   

                 